In [1]:
import os
import cv2
#use numpy >= 1.26.0 and <2.0

In [2]:
input_dir = 'Dataset/'  # Your dataset directory
output_dir = 'resized_dataset/'  # Directory to save resized images
target_size = (224, 224)

In [3]:
os.makedirs(output_dir, exist_ok=True)

In [4]:
for category in os.listdir(input_dir):
    category_path = os.path.join(input_dir, category)

    # Create a corresponding directory in the output folder
    output_category_dir = os.path.join(output_dir, category)
    os.makedirs(output_category_dir, exist_ok=True)

    # Check if it's a folder (for each class)
    if os.path.isdir(category_path):
        for filename in os.listdir(category_path):
            if filename.endswith(('.jpg', '.jpeg', '.png')):  # Filter for image files
                img_path = os.path.join(category_path, filename)
                
                # Read image using OpenCV
                img = cv2.imread(img_path)

                # Resize the image
                resized_img = cv2.resize(img, target_size)

                # Save resized image in the corresponding output folder
                output_img_path = os.path.join(output_category_dir, filename)
                cv2.imwrite(output_img_path, resized_img)

In [9]:
!pip install tensorflow

  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached wrapt-1.16.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached grpcio-1.66.2-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached wheel-0.44.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached rich-13.9.1-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 1.8/385.2 MB 10.1 MB/s eta 0:00:39
   ---------------------------------------- 4.5/385.2 MB 12.2 MB/s eta 0:00:32
    --------------------------------------- 6.8/385.2 MB 11.7 MB/s et

  You can safely remove it manually.
  You can safely remove it manually.


In [13]:
pip install --upgrade numpy

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", but you have numpy 2.1.1 which is incompatible.



  Using cached numpy-2.1.1-cp312-cp312-win_amd64.whl.metadata (59 kB)
Using cached numpy-2.1.1-cp312-cp312-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [12]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
resized_dataset_dir = 'resized_dataset/'
# Path to save augmented images
augmented_images_dir = 'augmented_dataset/'
os.makedirs(augmented_images_dir, exist_ok=True)

In [7]:
datagen = ImageDataGenerator(
    rotation_range=40,       # Randomly rotate images in the range (degrees)
    width_shift_range=0.2,   # Randomly translate images horizontally (20% of width)
    height_shift_range=0.2,  # Randomly translate images vertically (20% of height)
    shear_range=0.2,         # Shear intensity
    zoom_range=0.2,          # Randomly zoom into images
    horizontal_flip=True,     # Randomly flip images
    fill_mode='nearest'      # Fill strategy for newly created pixels
)

In [8]:
for class_name in os.listdir(resized_dataset_dir):
    class_path = os.path.join(resized_dataset_dir, class_name)

    # Create a corresponding directory in the output folder
    augmented_class_dir = os.path.join(augmented_images_dir, class_name)
    os.makedirs(augmented_class_dir, exist_ok=True)

    # Check if it's a folder (for each class)
    if os.path.isdir(class_path):
       
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)    
            # Read image using OpenCV
            img = cv2.imread(img_path)

            original_img_path = os.path.join(augmented_class_dir, img_file)
            cv2.imwrite(original_img_path, img)
            
            img_array = np.expand_dims(img, axis=0)
            
            for i, augmented_img in enumerate(datagen.flow(img_array, batch_size=1)):
                # Save augmented images with a unique name
                augmented_img_path = os.path.join(augmented_class_dir, f"augmented_{img_file.split('.')[0]}_{i}.jpg")
                cv2.imwrite(augmented_img_path, augmented_img[0])  # Save the augmented image
                        
                # Stop after saving a certain number of augmented images to avoid too many files
                if i >= 4:  # Change this number as per your requirement
                    break

In [9]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [10]:
try:
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
except Exception as e:
    print(f"Error loading VGG16: {e}")
    base_model = VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))  # Load without weights

In [11]:
base_model.trainable = False

# Create a new model on top of it
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes for Kohli, Federer, Messi
])

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Function to load images and convert to NumPy array
"""def load_images_from_folder(folder):
    images = []
    labels = []
    
    for class_name in os.listdir(folder):
        class_folder = os.path.join(folder, class_name)
        
        if os.path.isdir(class_folder):
            for img_file in os.listdir(class_folder):
                img_path = os.path.join(class_folder, img_file)
                
                # Check for .jpg and .png formats
                if img_file.endswith(('.jpg', '.png')):
                    # Load image using OpenCV
                    img = cv2.imread(img_path)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
                    img_resized = cv2.resize(img, (224, 224))  # Resize to 224x224
                    images.append(img_resized)
                    labels.append(class_name)
    
    return np.array(images), np.array(labels)"""

"def load_images_from_folder(folder):\n    images = []\n    labels = []\n    \n    for class_name in os.listdir(folder):\n        class_folder = os.path.join(folder, class_name)\n        \n        if os.path.isdir(class_folder):\n            for img_file in os.listdir(class_folder):\n                img_path = os.path.join(class_folder, img_file)\n                \n                # Check for .jpg and .png formats\n                if img_file.endswith(('.jpg', '.png')):\n                    # Load image using OpenCV\n                    img = cv2.imread(img_path)\n                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB\n                    img_resized = cv2.resize(img, (224, 224))  # Resize to 224x224\n                    images.append(img_resized)\n                    labels.append(class_name)\n    \n    return np.array(images), np.array(labels)"

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255)  # Only rescaling is applied

In [28]:
#X, y = load_images_from_folder('augmented_dataset/')

# Normalize pixel values
#X = X / 255.0

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_generator = train_datagen.flow_from_directory(
    'augmented_dataset/',  # Directory with augmented image folders
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 157 images belonging to 3 classes.


In [15]:
model.fit(train_generator, epochs=12, steps_per_epoch=train_generator.samples // 32)

Epoch 1/12


C:\Users\jimit_moontechnolabs\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.3549 - loss: 4.4565
Epoch 2/12
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4688 - loss: 1.3592
Epoch 3/12


C:\Users\jimit_moontechnolabs\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.4925 - loss: 1.3305
Epoch 4/12
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6875 - loss: 0.6161
Epoch 5/12
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.7491 - loss: 0.5516
Epoch 6/12
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9375 - loss: 0.2159
Epoch 7/12
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.9500 - loss: 0.1597
Epoch 8/12
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9655 - loss: 0.1183
Epoch 9/12
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.9702 - loss: 0.0937
Epoch 10/12
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 0.0543
Epoch 11/12
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 1.0000 - loss: 0.0442
Epoch 12/12
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 0.0377


In [19]:
from tensorflow.keras.preprocessing import image

In [22]:
img_path = 'messi.jpg'
#img_path = 'aa5f6a21b6f819a12108217f5602c1f1.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
predictions = model.predict(img_array)
class_index = np.argmax(predictions)
classes = ['Virat Kohli', 'Roger Federer', 'Lionel Messi']
print(f'The predicted class is: {classes[class_index]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
The predicted class is: Lionel Messi


## ------------------------------------------End Of project----------------------------------------------

In [30]:
# Convert labels to categorical (one-hot encoding)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y = lb.fit_transform(y)

In [33]:
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

In [37]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("X dtype:", X.dtype)
print("y dtype:", y.dtype)

Shape of X: (157, 224, 224, 3)
Shape of y: (157, 3)
X dtype: float32
y dtype: float32


In [41]:
print("Any NaN in X:", np.isnan(X).any())
print("Any infinite in X:", np.isinf(X).any())
print("Any NaN in y:", np.isnan(y).any())
print("Any infinite in y:", np.isinf(y).any())

Any NaN in X: False
Any infinite in X: False
Any NaN in y: False
Any infinite in y: False


In [42]:
# Fit the model
#train_generator = train_datagen.flow(X, y, batch_size=32)

In [43]:
# Fit the model using the generator
#model.fit(train_generator, epochs=10, steps_per_epoch=len(X) // 32)

model.fit(X[:10], y[:10], epochs=10, batch_size=32)


ValueError: object __array__ method not producing an array